In [20]:
import pandas as pd
import requests
from tqdm import tqdm
import time

In [21]:
apikey = "sk-73eAaaMAgCE1wFNy58A29d4849B447CaA2E287A04d7fBfAc"

In [22]:
def send_chat_message_up(industry, up="上游", retry=0):
    url = 'http://23.105.199.129:3002/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {apikey}',
        'Content-Type': 'application/json',
    }
    payload = {
        "model" : "gpt-4",
        "messages": [
            {
                "content": f"提供给我{industry}的{up}产品，只需要五个，并用‘、’隔开给我。注意，不要带回答的前缀",
                "role": "user"
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        response_data = response.json()
        # 处理响应数据
        return response_data['choices'][0]['message']['content']
    else:
        print("Request failed with status code:", response.status_code)
        if retry > 10:
            print("重试次数过多", industry)
            return None
        return send_chat_message_up(industry, up="上游", retry=retry + 1)

In [29]:
def send_chat_message_up(industry, up="下游", retry=0):
    url = 'http://23.105.199.129:3002/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {apikey}',
        'Content-Type': 'application/json',
    }
    payload = {
        "model" : "gpt-4",
        "messages": [
            {
                "content": f"提供给我{industry}的{up}产品，只需要五个，并用‘、’隔开给我。注意，不要带回答的前缀",
                "role": "user"
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    print(response)
    if response.status_code == 200:
        response_data = response.json()
        # 处理响应数据
        return response_data['choices'][0]['message']['content']
    else:
        print("Request failed with status code:", response.status_code)
        if retry > 10:
            print("重试次数过多", industry)
            return None
        return send_chat_message_up(industry, up="下游", retry=retry + 1)

In [ ]:
send_chat_message_up('房地产')

<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
<Response [401]>
Request failed with status code: 401
重试次数过多 房地产


In [ ]:
def fill_keywords(row):
    try:
        upstream_keywords = send_chat_message_up(row['关键词'])
        downstream_keywords = send_chat_message_down(row['关键词'])          
        row['上游关键词'] = upstream_keywords
        row['下游关键词'] = downstream_keywords
    except Exception as e:
        print(f"Error processing row: {e}")
    return row

In [ ]:
# Assuming df_temp is your DataFrame
# Create a tqdm instance to track progress
tqdm_instance = tqdm(total=len(df_temp), desc="Processing Rows", position=0, leave=True)

for i, row in df_temp.iterrows():
    df_temp.loc[i] = fill_keywords(row)
    
    # Save progress after processing each row
    df_temp.to_excel(r"C:\Users\10378\Desktop\7_test.xlsx", index=False)
    
    # Update the progress bar
    tqdm_instance.update(1)

# Close the tqdm instance
tqdm_instance.close()

### test

In [1]:
import pandas as pd
from tqdm import tqdm
import time

In [2]:

## 取前十个数据作为测试集
temp = pd.read_excel(io=r"C:\Users\TUNGEE\Desktop\产业链补充数据TEMP.xlsx")
# temp = temp .iloc[0:1001,:]
# display(temp)
# temp = df .iloc[0:51,:]
# display(temp)

In [3]:
def send_temp_up(industry) :
    time.sleep(2)
    return industry+"up"

def send_temp_down(industry) :
    time.sleep(2)
    return industry+"down"

def fill_keywords(row):
    try:
        upstream_keywords = send_temp_up(row['经营产品'])
        downstream_keywords = send_temp_down(row['经营产品'])          
        row['上游关键词'] = upstream_keywords
        row['下游关键词'] = downstream_keywords
    except Exception as e:
        print(f"Error processing row: {e}")
    return row

In [33]:
# Assuming df_temp is your DataFrame
# Create a tqdm instance to track progress
tqdm_instance = tqdm(total=len(temp), desc="Processing Rows", position=0, leave=True)

for i, row in temp.iterrows():
    try:
        temp.loc[i] = fill_keywords(row)
    except Exception as e:
        print(f"Error processing row: {e}")
    
    # Save progress after processing each row
    temp.to_excel(r"C:\Users\TUNGEE\Desktop\temp.xlsx", index=False)   
    
    # Update the progress bar
    tqdm_instance.update(1)

# Close the tqdm instance
tqdm_instance.close()

### 多线程

In [4]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# 创建tqdm 实例
tqdm_instance = tqdm(total=len(temp), desc="处理行", position=0, leave=True)

def process_row(i, row):
    try:
        # 填充关键词
        temp.loc[i] = fill_keywords(row)
    except Exception as e:
        print(f"填充时出错: {e}")

    # 每处理完一行就保存进度
    temp.to_excel(r"C:\Users\TUNGEE\Desktop\temp.xlsx", index=False)

    # 更新进度条
    tqdm_instance.update(1)

# 设置线程数
num_threads = 5  

# 使用 ThreadPoolExecutor 进行并发处理
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # 提交每一行的任务以实现并行处理
    futures = [executor.submit(process_row, i, row) for i, row in temp.iterrows()]

    # 等待所有任务完成
    for future in tqdm(futures, desc="等待线程", position=1, leave=True):
        future.result()

# 关闭 tqdm 实例
tqdm_instance.close()

处理行: 100%|██████████| 51/51 [00:44<00:00,  1.15it/s]


### logging && ThreadPool

In [1]:
import logging
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import time

## 取前十个数据作为测试集
temp = pd.read_excel(io=r"C:\Users\TUNGEE\Desktop\产业链补充数据TEMP.xlsx")
# temp = temp.iloc[0:1001,:]

def send_temp_up(industry) :
    time.sleep(2)
    return industry+"up"

def send_temp_down(industry) :
    time.sleep(2)
    return industry+"down"

def fill_keywords(row):
    try:
        upstream_keywords = send_temp_up(row['经营产品'])
        downstream_keywords = send_temp_down(row['经营产品'])          
        row['上游关键词'] = upstream_keywords
        row['下游关键词'] = downstream_keywords
    except Exception as e:
        print(f"Error processing row: {e}")
    return row

# 配置日志
logging.basicConfig(filename=r"C:\Users\TUNGEE\Desktop\fill_keywords.log", level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler()  # 创建一个控制台处理程序
logging.getLogger().addHandler(console_handler)  # 将控制台处理程序添加到根记录器

# 创建 tqdm 实例
tqdm_instance = tqdm(total=len(temp), desc="处理行", position=0, leave=True)

def process_row(i, row):
    try:
        # 填充关键词
        keywords_result = fill_keywords(row)
        temp.loc[i] = keywords_result
        # 记录填充关键词的信息到日志

        ## gpt输出包含道歉，则出现gpt回答问题出错
        if(keywords_result['上游关键词'].__contains__("对不起") or keywords_result['下游关键词'].__contains__('对不起')):
            logging.info(f"---gpt error: answer error--- 行 {i} 上游关键词结果: {row['上游关键词']} ||  下游关键词结果: {row['下游关键词']}")

        ## gpt输出包含冒号，则出现格式回答问题
        if(keywords_result['上游关键词'].__contains__("：") or keywords_result['下游关键词'].__contains__('：')):
            logging.info(f"---gpt error: prefigx error--- 行 {i} 上游关键词结果: {row['上游关键词']} || 下游关键词结果: {row['下游关键词']}")
    except Exception as e:
        # 记录异常信息到日志
        logging.error(f"填充时出错: {e}")

    # 每处理完一行就保存进度
    temp.to_excel(r"C:\Users\TUNGEE\Desktop\temp.xlsx", index=False)

    # 更新进度条
    tqdm_instance.update(1)

# 设置线程数
num_threads = 6

# 使用 ThreadPoolExecutor 进行并发处理
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # 提交每一行的任务以实现并行处理
    futures = [executor.submit(process_row, i, row) for i, row in temp.iterrows()]

    # 等待所有任务完成
    for future in tqdm(futures, desc="等待线程", position=1, leave=True):
        future.result()

# 关闭 tqdm 实例
tqdm_instance.close()


处理行:  69%|██████▊   | 4333/6310 [1:52:11<56:30,  1.72s/it]  